In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns

In [2]:
pd.set_option("display.max_columns", 50)

### Loading Kaggle IPL data

In [3]:
df = pd.read_csv("data/df_kaggleAllIpl_preprocessed.csv")
del df['Unnamed: 0']

#### Adding a is_wicket column

In [4]:
df["is_bowler_wicket"] = df["dismissal_kind"].isin(["caught","bowled","lbw","caught and bowled","stumped"]).values.astype(int)

In [196]:
df.head(2)

,match_id,season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,wide_runs,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed,dismissal_kind,fielder,is_bowler_wicket
0,1,2017,Hyderabad,2017-04-05,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Sunrisers Hyderabad,35,0,Yuvraj Singh,"Rajiv Gandhi International Stadium, Uppal",AY Dandekar,NJ Llong,NaN,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,1,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,0
1,1,2017,Hyderabad,2017-04-05,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Sunrisers Hyderabad,35,0,Yuvraj Singh,"Rajiv Gandhi International Stadium, Uppal",AY Dandekar,NJ Llong,NaN,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,2,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,0


### Player statistics: Season and venue wise

In [197]:
### Number of runs scored
dfp = df.groupby(['season','venue','batting_team','batsman']).sum()["batsman_runs"].reset_index()
dfp = dfp.sort_values(['season', 'venue', 'batting_team','batsman_runs'], ascending=False)
dfp.rename(columns={"batsman":"player","batting_team":"team","batsman_runs":"runs_scored"}, inplace=True)

### Number of balls faced 
df1 = df.copy()
df1 = df1[df1['is_super_over']==0]
df1 = df1[df1['wide_runs']==0]
df1 = df1[df1['noball_runs']==0]
df1 = df1.groupby(['season','venue','batting_team','batsman']).count()["date"].reset_index()
df1.rename(columns={"date":"balls_faced","batting_team":"team","batsman":"player"}, inplace=True)
dfp = dfp.set_index(['season','venue','team','player']).join(df1.set_index(['season','venue','team','player'])).reset_index()

### Number of innings
df1 = df.copy()
df1 = df1.drop_duplicates(['match_id','batsman'])
df1 = df1.groupby(['season','venue','batting_team','batsman']).count()["date"].reset_index()
df1.rename(columns={"batting_team":"team","date":"num_innings","batsman":"player"}, inplace=True)
dfp = dfp.set_index(['season','venue','team','player']).join(df1.set_index(['season','venue','team','player'])).reset_index()

### Number of wickets
df1 = df.copy()
df1 = df1.groupby(['season','venue','bowling_team','bowler']).sum()["is_bowler_wicket"].reset_index()
df1.rename(columns={"is_bowler_wicket":"wickets","bowling_team":"team","bowler":"player"}, inplace=True)
df1 = df1.sort_values(['season','wickets'], ascending=False)
dfp = dfp.set_index(['season','venue','team','player']).join(df1.set_index(['season','venue','team','player']), how="outer").reset_index()

### Number of balls bowled 
df1 = df.copy()
df1 = df1[df1['is_super_over']==0]
df1 = df1[df1['wide_runs']==0]
df1 = df1[df1['noball_runs']==0]
df1 = df1.groupby(['season','venue','bowling_team','bowler']).count()["date"].reset_index()
df1.rename(columns={"date":"balls_bowled","bowling_team":"team","bowler":"player"}, inplace=True)
dfp = dfp.set_index(['season','venue','team','player']).join(df1.set_index(['season','venue','team','player'])).reset_index()

### Number of runs conceded
df1 = df.copy()
df1 = df1[df1['bye_runs']==0]
df1 = df1[df1['legbye_runs']==0]
df1 = df1.groupby(['season','venue','bowling_team','bowler']).sum()["total_runs"].reset_index()
df1.rename(columns={"total_runs":"runs_conceded","bowling_team":"team","bowler":"player"}, inplace=True)
dfp = dfp.set_index(['season','venue','team','player']).join(df1.set_index(['season','venue','team','player'])).reset_index()

### For players who did not bowl, wickets->0 balls_bowled->0 runs_conceded->0
dfp["wickets"].fillna(0, inplace=True)
dfp["balls_bowled"].fillna(0, inplace=True)
dfp["runs_conceded"].fillna(0, inplace=True)

### For players who did not bat, runs_scored->0, balls_faced->0
dfp["runs_scored"].fillna(0, inplace=True)
dfp["balls_faced"].fillna(0, inplace=True)
dfp["num_innings"].fillna(0, inplace=True)

dfp.head(5)

,season,venue,team,player,runs_scored,balls_faced,num_innings,wickets,balls_bowled,runs_conceded
0,2008,Dr DY Patil Sports Academy,Chennai Super Kings,CK Kapugedera,8.0,11.0,1.0,0.0,0.0,0.0
1,2008,Dr DY Patil Sports Academy,Chennai Super Kings,JA Morkel,16.0,13.0,1.0,2.0,24.0,25.0
2,2008,Dr DY Patil Sports Academy,Chennai Super Kings,L Balaji,0.0,0.0,0.0,0.0,24.0,42.0
3,2008,Dr DY Patil Sports Academy,Chennai Super Kings,M Muralitharan,0.0,0.0,0.0,2.0,24.0,39.0
4,2008,Dr DY Patil Sports Academy,Chennai Super Kings,M Ntini,0.0,0.0,0.0,0.0,24.0,21.0


### Verify the player statistics

In [200]:
aa = dfp.groupby(['season','team','player']).sum()["wickets"].reset_index()
aa = aa[aa["season"]==2016]
aa.sort_values("wickets", ascending=False).head(3)

,season,team,player,wickets
1527,2016,Sunrisers Hyderabad,B Kumar,23.0
1523,2016,Royal Challengers Bangalore,YS Chahal,21.0
1516,2016,Royal Challengers Bangalore,SR Watson,20.0


### Derived statistics
#### Batting: Average, strike-rate
#### Bowling:Average, strike-rate, economy rate

In [201]:
### Batting
dfp["bat_avg"] = dfp["runs_scored"]/dfp["num_innings"]
dfp["bat_sr"]  = dfp["runs_scored"]/dfp["balls_faced"]*100
dfp["bat_avgsr"]= dfp["bat_avg"]*dfp["bat_sr"]/100

### Bowling
dfp["bowl_avg"]= dfp["runs_conceded"]/dfp["wickets"]
dfp["bowl_sr"] = dfp["balls_bowled"]/dfp["wickets"]
dfp["bowl_econ"]=dfp["runs_conceded"]/dfp["balls_bowled"]*6
dfp.head(3)

,season,venue,team,player,runs_scored,balls_faced,num_innings,wickets,balls_bowled,runs_conceded,bat_avg,bat_sr,bat_avgsr,bowl_avg,bowl_sr,bowl_econ
0,2008,Dr DY Patil Sports Academy,Chennai Super Kings,CK Kapugedera,8.0,11.0,1.0,0.0,0.0,0.0,8.0,72.727273,5.818182,NaN,NaN,NaN
1,2008,Dr DY Patil Sports Academy,Chennai Super Kings,JA Morkel,16.0,13.0,1.0,2.0,24.0,25.0,16.0,123.076923,19.692308,12.500000,12.000000,6.25
2,2008,Dr DY Patil Sports Academy,Chennai Super Kings,L Balaji,0.0,0.0,0.0,0.0,24.0,42.0,NaN,NaN,NaN,inf,inf,10.50


### Per season venue wise statistics: to make player statistics venue and season (=>pitch) neutral

In [202]:
### Median bat_avg * bat_sr for each venue during different seasons
dfv = dfp.copy()
dfv = dfv[dfv["runs_scored"]>=20]
dfv = dfv.groupby(["season","venue"]).median()[["bat_avg","bat_sr","bat_avgsr"]].reset_index()
dfv.rename(columns={"bat_avg":"sv_bat_avg","bat_sr":"sv_bat_sr", "bat_avgsr":"sv_bat_avgsr"}, inplace=True)

### Standard deviation: batting average, batting strike rate
df1 = dfp.copy()
df1 = df1[df1["runs_scored"]>=20]
df1 = df1.groupby(["season","venue"]).std()[["bat_avg","bat_sr"]].reset_index()
df1.rename(columns={"bat_avg":"sv_std_bat_avg","bat_sr":"sv_std_bat_sr"}, inplace=True)

dfv = dfv.set_index(["season","venue"]).join(df1.set_index(["season","venue"])).reset_index()

### Median bowling average, bowling economy for each venue
df1 = dfp.copy()
df1 = df1[np.isfinite(df1["bowl_avg"])]
df1 = df1[np.isfinite(df1["bowl_sr"])]
df1 = df1[np.isfinite(df1["bowl_econ"])]
df1 = df1[df1["balls_bowled"]>=12]
df1 = df1.groupby(["season","venue"]).median()[["bowl_avg","bowl_econ"]].reset_index()

dfv = dfv.set_index(["season","venue"]).join(df1.set_index(["season","venue"])).reset_index()
dfv.rename(columns={"bowl_avg":"sv_bowl_avg", "bowl_econ":"sv_bowl_econ","bowl_avgecon":"sv_bowl_avgecon"}, inplace=True)

### Standard deviation: bowling average, bowling economy for each venue
df1 = dfp.copy()
df1 = df1[np.isfinite(df1["bowl_avg"])]
df1 = df1[np.isfinite(df1["bowl_sr"])]
df1 = df1[np.isfinite(df1["bowl_econ"])]
df1 = df1[df1["balls_bowled"]>=12]
df1 = df1.groupby(["season","venue"]).std()[["bowl_avg","bowl_econ"]].reset_index()
df1.rename(columns={"bowl_avg":"sv_std_bowl_avg","bowl_econ":"sv_std_bowl_econ"}, inplace=True)

dfv = dfv.set_index(["season","venue"]).join(df1.set_index(["season","venue"])).reset_index()
dfv.head(3)

,season,venue,sv_bat_avg,sv_bat_sr,sv_bat_avgsr,sv_std_bat_avg,sv_std_bat_sr,sv_bowl_avg,sv_bowl_econ,sv_std_bowl_avg,sv_std_bowl_econ
0,2008,Dr DY Patil Sports Academy,29.0,137.777778,40.080357,20.252371,33.292177,18.333333,7.500000,13.588289,1.844588
1,2008,Eden Gardens,26.0,121.127503,29.250919,15.291004,42.184812,19.333333,7.000000,14.297174,2.158578
2,2008,Feroz Shah Kotla,30.5,160.622711,56.227508,13.556141,44.202442,22.750000,8.433333,27.485175,1.594114


### Merge player and venue statistics

In [203]:
dfpv = dfp.set_index(["season","venue"]).join(dfv.set_index(["season","venue"])).reset_index()
dfpv = dfpv[["season","venue","team","player","runs_scored","bat_avg","bat_sr","bat_avgsr",
             "sv_bat_avg","sv_bat_sr","sv_bat_avgsr","sv_std_bat_avg","sv_std_bat_sr", "wickets",
             "balls_bowled","bowl_avg","bowl_sr","bowl_econ","sv_bowl_avg","sv_bowl_econ",
             "sv_std_bowl_avg","sv_std_bowl_econ"]]
dfpv.head(3)

,season,venue,team,player,runs_scored,bat_avg,bat_sr,bat_avgsr,sv_bat_avg,sv_bat_sr,sv_bat_avgsr,sv_std_bat_avg,sv_std_bat_sr,wickets,balls_bowled,bowl_avg,bowl_sr,bowl_econ,sv_bowl_avg,sv_bowl_econ,sv_std_bowl_avg,sv_std_bowl_econ
0,2008,Dr DY Patil Sports Academy,Chennai Super Kings,CK Kapugedera,8.0,8.0,72.727273,5.818182,29.0,137.777778,40.080357,20.252371,33.292177,0.0,0.0,NaN,NaN,NaN,18.333333,7.5,13.588289,1.844588
1,2008,Dr DY Patil Sports Academy,Chennai Super Kings,JA Morkel,16.0,16.0,123.076923,19.692308,29.0,137.777778,40.080357,20.252371,33.292177,2.0,24.0,12.500000,12.000000,6.25,18.333333,7.5,13.588289,1.844588
2,2008,Dr DY Patil Sports Academy,Chennai Super Kings,L Balaji,0.0,NaN,NaN,NaN,29.0,137.777778,40.080357,20.252371,33.292177,0.0,24.0,inf,inf,10.50,18.333333,7.5,13.588289,1.844588


### Batting metrics:
For each season
    1. WPA
    2. sum_over_venue ( bat_avg * bat_sr - (bat_avg_venueSeason * bat_sr_venueSeason) )
                                   or
       sum_over_venue ( (bat_avg - bat_avg_venueSeason) * (bat_sr - bat_sr_venueSeason) )
    3. (?) Total runs, average, strike rate

### Batting metric 2

In [204]:
dfbat = dfpv.copy()

dfbat = dfbat[dfbat["runs_scored"]>=20]
dfbat.loc[dfbat["runs_scored"]==0,"bat_avg"] = dfbat.loc[dfbat["runs_scored"]==0,"sv_bat_avg"]
dfbat.loc[dfbat["runs_scored"]==0,"bat_sr"]  = dfbat.loc[dfbat["runs_scored"]==0,"sv_bat_sr"]

dfbat["zs_bat_avg"] = dfbat["bat_avg"]-dfbat["sv_bat_avg"]/dfbat["sv_std_bat_avg"]
dfbat["zs_bat_sr"]  = dfbat["bat_sr"]-dfbat["sv_bat_sr"]/dfbat["sv_std_bat_sr"]
dfbat = dfbat.groupby(["season","player"]).sum()[["runs_scored","zs_bat_avg","zs_bat_sr"]].reset_index()

### Weights
wr = 2; wa=2; ws=1
dfbat["bat_venue"] = wr*dfbat["runs_scored"] + wa*dfbat["zs_bat_avg"]  +  ws*dfbat["zs_bat_sr"]

In [205]:
aa = dfbat[(dfbat["season"]==2016)]
aa.sort_values("bat_venue", ascending=False).head(5)

,season,player,runs_scored,zs_bat_avg,zs_bat_sr,bat_venue
799,2016,V Kohli,959.0,416.951454,900.139851,3652.042759
735,2016,DA Warner,830.0,387.256752,1010.560249,3445.073753
721,2016,AB de Villiers,681.0,332.581792,1115.871252,3143.034835
779,2016,RG Sharma,484.0,330.790444,933.666216,2563.247105
723,2016,AJ Finch,393.0,309.472470,1142.578065,2547.523005


### Bowling metrics:

    1. WPA
    2. sum_over_venue ( bowl_avg * bowl_econ - (bowl_avg_venueSeason * bowl_econ_venueSeason) )
                                   or
       sum_over_venue( (bowl_avg - bowl_avg_venueSeason) * (bowl_econ - bowl_econ_venueSeason) )       
    3. Wickets weighted by batsman's metric.

### Bowling metric 2

In [206]:
dfball=dfpv.copy()

dfball.loc[dfball["wickets"]==0,"bowl_avg"] = dfball.loc[dfball["wickets"]==0,"sv_bowl_avg"]
dfball = dfball[dfball["balls_bowled"]>=12]

dfball["zs_bowl_avg"] = dfball["sv_bowl_avg"]-dfball["bowl_avg"]/dfball["sv_std_bowl_avg"]
dfball["zs_bowl_econ"] =dfball["sv_bowl_econ"]-dfball["bowl_econ"]/dfball["sv_std_bowl_econ"]
dfball = dfball.groupby(["season","player"]).sum()[["wickets","zs_bowl_avg","zs_bowl_econ"]].reset_index()

### Weights
ww = 20; wa=1; we=1
dfball["bowl_venue"] = ww*dfball["wickets"] + wa*dfball["zs_bowl_avg"]  +  we*dfball["zs_bowl_econ"]

In [207]:
aa = dfball[dfball["season"]==2016]
aa.sort_values("bowl_venue", ascending=False).head(5)

,season,player,wickets,zs_bowl_avg,zs_bowl_econ,bowl_venue
871,2016,B Kumar,23.0,163.456709,28.716446,652.173155
948,2016,SR Watson,20.0,169.038922,29.256642,598.295564
885,2016,DS Kulkarni,18.0,182.736454,36.796106,579.532560
881,2016,DJ Bravo,17.0,197.840421,34.720227,572.560647
917,2016,MJ McClenaghan,16.0,192.535857,33.579684,546.115541
